## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
## 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Murdochville,CA,48.9583,-65.5006,24.30,77,92,24.00,overcast clouds
1,1,Qaanaaq,GL,77.4840,-69.3632,-4.02,68,56,3.33,broken clouds
2,2,Hermanus,ZA,-34.4187,19.2345,67.89,78,100,10.71,overcast clouds
3,3,Mataura,NZ,-46.1927,168.8643,48.47,94,82,1.03,broken clouds
4,4,Bethel,US,41.3712,-73.4140,47.43,38,1,4.00,clear sky


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) 
                                    & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(30)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Mahebourg,MU,-20.4081,57.7000,79.09,69,40,11.50,scattered clouds
8,8,Castro,BR,-24.7911,-50.0119,77.88,44,94,5.41,overcast clouds
10,10,Hobyo,SO,5.3505,48.5268,79.05,80,61,19.39,broken clouds
12,12,Rikitea,PF,-23.1203,-134.9692,77.49,75,72,9.28,broken clouds
13,13,Riohacha,CO,11.5444,-72.9072,86.63,55,98,14.00,overcast clouds
22,22,Sur,OM,22.5667,59.5289,72.34,57,2,7.81,clear sky
23,23,Samarai,PG,-10.6167,150.6667,81.77,81,100,12.62,overcast clouds
24,24,Sao Filipe,CV,14.8961,-24.4956,75.65,69,20,9.95,few clouds
25,25,Bandarbeyla,SO,9.4942,50.8122,77.11,80,6,24.07,clear sky
27,27,Georgetown,MY,5.4112,100.3354,81.90,80,40,8.05,scattered clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                251
City                   251
Country                250
Lat                    251
Lng                    251
Max Temp               251
Humidity               251
Cloudiness             251
Wind Speed             251
Current Description    251
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.count()

City_ID                250
City                   250
Country                250
Lat                    250
Lng                    250
Max Temp               250
Humidity               250
Cloudiness             250
Wind Speed             250
Current Description    250
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(30)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Mahebourg,MU,79.09,scattered clouds,-20.4081,57.7000,
8,Castro,BR,77.88,overcast clouds,-24.7911,-50.0119,
10,Hobyo,SO,79.05,broken clouds,5.3505,48.5268,
12,Rikitea,PF,77.49,broken clouds,-23.1203,-134.9692,
13,Riohacha,CO,86.63,overcast clouds,11.5444,-72.9072,
22,Sur,OM,72.34,clear sky,22.5667,59.5289,
23,Samarai,PG,81.77,overcast clouds,-10.6167,150.6667,
24,Sao Filipe,CV,75.65,few clouds,14.8961,-24.4956,
25,Bandarbeyla,SO,77.11,clear sky,9.4942,50.8122,
27,Georgetown,MY,81.90,scattered clouds,5.4112,100.3354,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row['Lat']
    lng=row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print('Hotel not found... skipping')
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
clean_hotel_df.count()

City                   225
Country                225
Max Temp               225
Current Description    225
Lat                    225
Lng                    225
Hotel Name             225
dtype: int64

In [15]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))